Question 1 Part (a)

In [1]:
import numpy as np
import scipy.stats as sts
import statistics 
import pandas as pd

# Load in and display the data
#macroseries = np.loadtxt('MacroSeries.txt', dtype={'names': ('c_t', 'k_t', 'w_t', 'r_t'), 'formats': ('|S15', '|S15', '|S15', '|S15')}, delimiter = ',')
macroseries = pd.read_csv('Macroseries.csv', header=0)
#print(macroseries)
#macroseries.shift(periods=1)
#print(macroseries['k_t'])


beta = 0.99

def eq_3(w_t, k_t, alpha):

    z_t = np.log(w_t/((1-alpha)*k_t**alpha))

    return z_t

def norm_pdf_z(z_t, rho, sigma, mu):

    z_t_1 = z_t.shift(1)
    z_t_1[0] = mu
    pdf_vals    = (1/(sigma * np.sqrt(2 * np.pi)) *
                    np.exp( - (z_t - rho*z_t_1 - (1-rho)*mu)**2 / (2 * sigma**2)))
    
    return pdf_vals

def log_lik_norm(z_t, rho, sigma, mu):
    
    pdf_vals = norm_pdf_z(z_t, rho, sigma, mu)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    
    return log_lik_val

def crit(params, *args):
    alpha, rho, sigma, mu = params
    w_t, k_t = args
    z_t = eq_3(w_t, k_t, alpha)
    log_lik_val = log_lik_norm(z_t, rho, sigma, mu)
    neg_log_lik_val = -log_lik_val
    
    return neg_log_lik_val

# Set up initial guesses
alpha_0 = 0.5
rho_0 = 0
sigma_0 = 0.5
mu_0 = 0.5

# 0.92199196448 rho_MLE= 0.99999999 mu_MLE= 0.8384721751 sigma_MLE= 0.379922805263
#print(crit(np.array([0.92199196448,  0.999999999999,  0.379922805263, 0.8384721751]), (macroseries['w_t'],macroseries['cap_t'])))

print('Initial alpha_0=', alpha_0, 'Initial rho_0=', rho_0, 'Initial sigma_0=', sigma_0, 'Initial mu_0=', mu_0)

import scipy.optimize as opt

mle_args = (macroseries['w_t'], macroseries['cap_t'])
params_init = np.array([alpha_0,rho_0,sigma_0,mu_0])
results = opt.minimize(crit, params_init, args=(mle_args), bounds = ((1e-10, 1-1e-10), (-1+1e-10, 1-1e-10), (1e-10, None), (1e-10, None)), method = 'L-BFGS-B')
#minimizer_kwargs = dict(method="L-BFGS-B", 
#                       bounds= ((1e-8, 1-1e-8), (-1+1e-8, 1-1e-8), (1e-8, None), (1e-8, None)),
#                       args = mle_args)
#results = opt.basinhopping(crit, params_init, minimizer_kwargs= minimizer_kwargs)
print(results)

alpha_MLE, rho_MLE, sigma_MLE, mu_MLE = results.x
inv_hess = results.hess_inv*np.identity(4) #([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
max_log_lik = log_lik_norm(eq_3(macroseries['w_t'], macroseries['cap_t'], alpha_MLE),rho_MLE,sigma_MLE,mu_MLE)
print('alpha_MLE=', alpha_MLE, ' rho_MLE=', rho_MLE, 'sigma_MLE=', sigma_MLE, 'mu_MLE', mu_MLE, 'log Likelihood=', max_log_lik)
print('Inverse Hessian=', inv_hess)

Initial alpha_0= 0.5 Initial rho_0= 0 Initial sigma_0= 0.5 Initial mu_0= 0.5
      fun: 36.063361542009574
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-64.24465795,   3.13432764,  47.65166963, -22.37200363])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 105
      nit: 12
   status: 0
  success: True
        x: array([ 0.92199195,  1.        ,  0.37992366,  0.83847207])
alpha_MLE= 0.921991951236  rho_MLE= 0.9999999999 sigma_MLE= 0.379923660767 mu_MLE 0.838472073256 log Likelihood= -36.063361542009574
Inverse Hessian= [[ 0.01692001 -0.00321062 -0.02074741  0.00279938]
 [-0.00321062  0.0167945   0.02770204 -0.05572709]
 [-0.02074741  0.02770204  0.2195734  -0.44003845]
 [ 0.00279938 -0.05572709 -0.44003845  1.04533837]]


Question 1 Part (b)

In [2]:
def eq_4(r_t, k_t, alpha):

    z_t = np.log(r_t/(alpha*k_t**(alpha-1)))

    return z_t

def crit_b(params, *args):
    alpha, rho, sigma, mu = params
    r_t, k_t = args
    z_t = eq_4(r_t, k_t, alpha)
    log_lik_val = log_lik_norm(z_t, rho, sigma, mu)
    neg_log_lik_val = -log_lik_val
    
    return neg_log_lik_val

# Set up initial guesses
alpha_0_b = 0.5
rho_0_b = 0.5
sigma_0_b = 1
mu_0_b = 1

print('Initial alpha_0=', alpha_0_b, 'Initial rho_0=', rho_0_b, 'Initial sigma_0=', sigma_0_b, 'Initial mu_0=', mu_0_b)

mle_args_b = (macroseries['r_t'], macroseries['cap_t'])
params_init_b = np.array([alpha_0_b,rho_0_b,sigma_0_b,mu_0_b])
results_b = opt.minimize(crit_b, params_init_b, args=(mle_args_b), bounds = ((1e-10, 1-1e-10), (-1+1e-10, 1-1e-10), (1e-10, None), (1e-10, None)), method = 'L-BFGS-B', options ={'ftol': 1e-10})
#minimizer_kwargs_b = dict(method="L-BFGS-B", 
#                       bounds= ((1e-8, 1-1e-8), (-1+1e-8, 1-1e-8), (1e-8, None), (1e-8, None)),
#                       args = mle_args_b)
#results_b = opt.basinhopping(crit_b, params_init_b, minimizer_kwargs= minimizer_kwargs_b)
print(results_b)

alpha_MLE_b, rho_MLE_b, sigma_MLE_b, mu_MLE_b = results_b.x
inv_hess_b = results_b.hess_inv*np.identity(4) #([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
max_log_lik_b = log_lik_norm(eq_4(macroseries['r_t'], macroseries['cap_t'], alpha_MLE_b),rho_MLE_b,sigma_MLE_b,mu_MLE_b)
print('alpha_MLE=', alpha_MLE_b, ' rho_MLE=', rho_MLE_b, 'sigma_MLE=', sigma_MLE_b, 'mu_MLE', mu_MLE_b, 'log Likelihood=', max_log_lik_b)
print('Inverse Hessian=', inv_hess_b)

Initial alpha_0= 0.5 Initial rho_0= 0.5 Initial sigma_0= 1 Initial mu_0= 1
      fun: 80.34156413730658
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  75.03204387,  -19.02322566,  108.59332633,    4.4036895 ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 35
      nit: 5
   status: 0
  success: True
        x: array([ 1.        ,  0.84963564,  0.86364662,  1.01055912])
alpha_MLE= 0.9999999999  rho_MLE= 0.849635638864 sigma_MLE= 0.863646623473 mu_MLE 1.01055911652 log Likelihood= -80.34156413730658
Inverse Hessian= [[  0.60334647   1.06964833  -3.33570027  -0.49556181]
 [  1.06964833   2.51429816  -6.36351169  -0.83113548]
 [ -3.33570027  -6.36351169  18.84081891   2.46169128]
 [ -0.49556181  -0.83113548   2.46169128   1.33782384]]


Question 1 Part (c)

In [3]:
z_star = eq_4(1,7500000,alpha_MLE)
print('z*=', z_star)

def norm_pdf_z_c(z_t, rho, sigma, mu):

    pdf_vals    = (1/(sigma * np.sqrt(2 * np.pi)) *
                    np.exp( - (z_t - rho*10 - (1-rho)*mu)**2 / (2 * sigma**2)))
    
    return pdf_vals

import scipy.integrate as integrate

prob_r_1 = 1 - integrate.quad(lambda x: norm_pdf_z_c(x, rho_MLE, sigma_MLE, mu_MLE), 0, z_star)[0]
print('Probability that the interest rate this period will be greater than 1:', prob_r_1)

z*= 1.31611845953
Probability that the interest rate this period will be greater than 1: 1.0
